In [ ]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'h2_dashboard.settings')
django.setup()

In [2]:
from django.apps import apps

def get_benchmark_data():
    # List of benchmark models you have, e.g., ['ElectricityProductionBenchmark', 'CO2EmissionsBenchmark']
    benchmark_models = ['ElectricityProductionBenchmark', 'CO2EmissionsBenchmark']
    benchmark_data = {}

    for model_name in benchmark_models:
        model = apps.get_model('h2_prod', model_name)  # Adjust 'h2_prod' to your app name if different
        benchmarks = model.objects.all()

        for benchmark in benchmarks:
            key = f"{model_name}-{benchmark.year}"
            if key not in benchmark_data:
                benchmark_data[key] = []
            benchmark_data[key].append(benchmark.value)

    return benchmark_data


In [5]:
def current_total_co2_emissions(total_h2_production):
    # Percentage of hydrogen production from each source
    percentage_gas = 0.48
    percentage_oil = 0.3
    percentage_coal = 0.18
    percentage_electro = 0.04

    # CO2 emissions (kg) from each source
    co2_per_kg_gas = 12  # kg CO2e per kg H2
    co2_per_kg_oil = 18  # kg CO2e per kg H2
    co2_per_kg_coal = 24  # kg CO2e per kg H2
    co2_per_kg_electro = 22  # kg CO2e per kg H2

    # Convert total hydrogen production from megatonnes to kilograms (1 megatonne = 1,000,000,000 kg)
    total_h2_prod_kg = total_h2_production * 1e9

    # Calculating the CO2 emissions from each source
    co2_emissions_gas = total_h2_prod_kg * percentage_gas * co2_per_kg_gas
    co2_emissions_oil = total_h2_prod_kg * percentage_oil * co2_per_kg_oil
    co2_emissions_coal = total_h2_prod_kg * percentage_coal * co2_per_kg_coal
    co2_emissions_electro = total_h2_prod_kg * percentage_electro * co2_per_kg_electro

    # Summing up the CO2 emissions from all sources to get the total CO2 emissions from hydrogen production
    previous_co2_emission = co2_emissions_gas + co2_emissions_oil + co2_emissions_coal + co2_emissions_electro
    return previous_co2_emission * 1e-9  # Conversion to Mega tonnes

print(round(current_total_co2_emissions(120)))

1963


Customize Your Templates:

Begin integrating the AdminLTE template into your Django views. Replace the HTML structure in your Django templates with the structure from AdminLTE's HTML files, making sure to keep Django template tags and context variables intac

unit font size
move reset button to top left
remove calculate button
slider to stay in place, only return to default place when reset button is clicked

In [5]:
electrolyzer_rating = 20 # MW 
kwh_per_kg_h2 = 33.3 # kWh 
default_total_h2_production = 120  # megatonnes per year
electrolyzer_efficiency = 70 # percent
total_h2_production = 90 # megatonnes per year


electricity_per_kg_h2 = kwh_per_kg_h2 / (electrolyzer_efficiency / 100)
total_electricity_requirement = electricity_per_kg_h2 * total_h2_production * 1e9  # Conversion to kg
electrolyzer_consumption = electrolyzer_rating * 8760 # per year running 24/7
required_electrolyzer_units = (total_electricity_requirement*1e-3)/(electrolyzer_rating * 8760) 

print(round(total_electricity_requirement))
print(round(electrolyzer_consumption))
print(required_electrolyzer_units)

4281428571429
175200
24437.37769080235


In [5]:
post_data = {
    'csrfmiddlewaretoken': 'some_token',
    'hiddenH2Production': '152',
    'hiddenElectrolyzerEff': '70',
    'hiddenRenewablePct': '30',
    'hiddenCO2Emission': '0.47'
}



<class 'str'>


In [6]:
def convert_str_float(post_data):



    # Define the keys you expect to convert to float
    keys_to_convert = ['hiddenH2Production', 'hiddenElectrolyzerEff', 'hiddenRenewablePct', 'hiddenCO2Emission']

    # Create a dictionary to hold the converted values
    converted_values = {}

    # Iterate over the keys and convert the values
    for key in keys_to_convert:
        try:
            converted_values[key] = float(post_data.get(key, 0))
        except ValueError:
            # Handle the case where conversion fails
            print(f"Value conversion failed for key: {key}")
            converted_values[key] = 0  # or handle this differently as per your need

    # Logging the converted values and their types
    for key, value in converted_values.items():
        print(f"{key}: {value}, Type: {type(value)}")
    
    return converted_values

data = convert_str_float(post_data)
print(data)

In [8]:
def post_floater(post_data):
    for key, value in post_data.items():
        if key != 'csrfmiddlewaretoken':
            try:
                post_data[key] = float(value)
            except ValueError:
                # Handle the case where conversion fails
                print(f"Value conversion failed for key: {key}")
                post_data[key] = 0  # or handle this differently as per your need

    # Logging the converted values and their types
    for key, value in post_data.items():
        print(f"{key}: {value}, Type: {type(value)}")
    
    return post_data

print(post_floater(post_data))

{'csrfmiddlewaretoken': 'some_token', 'hiddenH2Production': 152.0, 'hiddenElectrolyzerEff': 70.0, 'hiddenRenewablePct': 30.0, 'hiddenCO2Emission': 0.47}


In [ ]:
def hydrogen_production_view(request):
    context = {'form': HydrogenProductionForm()}
    
    if request.method == 'POST':
        post_data = request.POST.copy()
        post_data = post_floater(post_data)
        
        if 'reset' in request.POST:
            # if reset button is clicked
            # code to present default value in context
            return render(request, 'h2_prod/hydrogen_production.html', context)
        else:
            # initially it was for if calculate button is clicked
            form = HydrogenProductionForm(post_data)
            if form.is_valid():
                # code to return calculation value and plot in context
                return render(request, 'h2_prod/hydrogen_production.html', context)
            else:
                print('form is not valid')
                context = {'form': form}
                
                    # Render the template with context
    return render(request, 'h2_prod/hydrogen_production.html', context)
                